In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt

from sklearn.metrics import mean_squared_error, mean_absolute_error, f1_score, make_scorer
from sklearn.linear_model import Ridge
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.decomposition import PCA

from sklearn.feature_selection import SelectKBest, f_regression

In [2]:
np.random.seed(7)

# metrics

In [3]:
def mae(y_true, y_pred) :
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    
    y_true = y_true.reshape(1, -1)[0]
    y_pred = y_pred.reshape(1, -1)[0]
    over_threshold = y_true >= 0.1
    
    return np.mean(np.abs(y_true[over_threshold] - y_pred[over_threshold]))

def fscore(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    
    y_true = y_true.reshape(1, -1)[0]
    y_pred = y_pred.reshape(1, -1)[0]
    remove_NAs = y_true >= 0
    
    y_true = np.where(y_true[remove_NAs] >= 0.1, 1, 0)
    y_pred = np.where(y_pred[remove_NAs] >= 0.1, 1, 0)
    
    return (f1_score(y_true, y_pred))

def maeOverFscore(y_true, y_pred):
    return mae(y_true, y_pred) / (fscore(y_true, y_pred) + 1e-07)

def score(y_val, pred):
    f_value = fscore(y_val, pred)
    mae_value = maeOverFscore(y_val, pred)
    
    print(f"fscore : {f_value} \t maeOverFscore : {mae_value}")
    
    return (f_value, mae_value)

# fscore_sklearn = make_scorer(fscore)
# maeOverFscore_sklearn = make_scorer(maeOverFscore)

# data load

In [4]:
base = os.getcwd()
data_path = os.path.join(base, 'data')
submit_path = os.path.join(base, 'submit')

def load(name):
    return np.load(os.path.join(data_path, f"{name}.npy"))

def reshape(data):
    return data.reshape(data.shape[0] * 40 * 40, data.shape[-1])

In [5]:
data = load('train')

# data seperate

In [6]:
x_train, x_val, y_train, y_val = train_test_split(data[:, :-1], data[:, -1], \
                                                  test_size=0.2, random_state=7)

del data
print(x_train.shape, x_val.shape)

(97286587, 14) (24321647, 14)


## Select K

In [7]:
scalers = [StandardScaler, RobustScaler, PCA]
names = ['Standard', 'Robust', 'PCA']
scores = dict()
features = dict()

In [10]:
for name, f_scaler in zip(names, scalers):
    scores[name] = dict()
    features[name] = dict()

    for i in [6, 7, 8, 9]:
        clf = Ridge(alpha=10.0)
        
        scaler = f_scaler()
        transformed_x = scaler.fit_transform(x_train)
        
        selectK = SelectKBest(score_func=f_regression, k=i)
        transformed_x = selectK.fit_transform(transformed_x, y_train)
        
        clf.fit(transformed_x, y_train)
        del transformed_x
        
        transformed_x = scaler.transform(x_val)
        transformed_x = selectK.transform(transformed_x)
        
        pred = clf.predict(transformed_x)
        
        f_value, mae_value = score(y_val, pred)
        
        scores[name][i] = [f_value, mae_value]
        features[name][i] = selectK.get_support()

fscore : 0.26333129273321404 	 maeOverFscore : 6.577648879807171
fscore : 0.2546803169128348 	 maeOverFscore : 6.7721025804650985


C:\Users\user\Anaconda3\lib\site-packages\sklearn\linear_model\ridge.py:147: LinAlgWarning: Ill-conditioned matrix (rcond=4.47911e-08): result may not be accurate.
  overwrite_a=True).T


fscore : 0.25463029222094796 	 maeOverFscore : 6.773522447575874


C:\Users\user\Anaconda3\lib\site-packages\sklearn\linear_model\ridge.py:147: LinAlgWarning: Ill-conditioned matrix (rcond=3.89889e-08): result may not be accurate.
  overwrite_a=True).T


fscore : 0.23854296442028572 	 maeOverFscore : 6.802608658905047
fscore : 0.2622524261765739 	 maeOverFscore : 6.604275575289949
fscore : 0.25430110974343756 	 maeOverFscore : 6.78213954975852
fscore : 0.25454167568999303 	 maeOverFscore : 6.775671716461214
fscore : 0.23851793072508642 	 maeOverFscore : 6.804616587953087
fscore : 0.23565206326288407 	 maeOverFscore : 7.068881568989658
fscore : 0.24100081408631271 	 maeOverFscore : 6.742727854122177
fscore : 0.2267925071124892 	 maeOverFscore : 7.164313418058886
fscore : 0.2274215024996598 	 maeOverFscore : 7.201747160420222


In [11]:
for name in names:
    print(name)
    for i in [6, 7, 8, 9]:
        print(f"fscore : {scores[name][i][0]} \t maeOverFscore : {scores[name][i][1]}")
        print(features[name][i])

Standard
fscore : 0.26333129273321404 	 maeOverFscore : 6.577648879807171
[False False  True  True  True  True  True  True False False False False
 False False]
fscore : 0.2546803169128348 	 maeOverFscore : 6.7721025804650985
[False False  True  True  True  True  True  True False False False False
 False  True]
fscore : 0.25463029222094796 	 maeOverFscore : 6.773522447575874
[False False  True  True  True  True  True  True False False False  True
 False  True]
fscore : 0.23854296442028572 	 maeOverFscore : 6.802608658905047
[False  True  True  True  True  True  True  True False False False  True
 False  True]
Robust
fscore : 0.2622524261765739 	 maeOverFscore : 6.604275575289949
[False False  True  True  True  True  True  True False False False False
 False False]
fscore : 0.25430110974343756 	 maeOverFscore : 6.78213954975852
[False False  True  True  True  True  True  True False False False False
 False  True]
fscore : 0.25454167568999303 	 maeOverFscore : 6.775671716461214
[False Fa

# Ridge alpha 12.5 SelectK 8
- fscore : 0.25464065823329085 	 maeOverFscore : 6.752701127612951
- fscore : 0.25458534451407944 	 maeOverFscore : 6.7528914220204515
- fscore : 0.2545837754161434 	 maeOverFscore : 6.702930965930638
- fscore : 0.25427823328070515 	 maeOverFscore : 6.747219104960767

# Ridge alpha 10.0 SelectK 8
- fscore : 0.25464065823329085 	 maeOverFscore : 6.752701129278926
- fscore : 0.25458534451407944 	 maeOverFscore : 6.75289142477857
- fscore : 0.2545837754161434 	 maeOverFscore : 6.70293096792134
- fscore : 0.25427826739578746 	 maeOverFscore : 6.747218201741558

# Ridge alpha 10.0 SelectK 8 StandardScaler
- fscore : 0.2546402018844707 	 maeOverFscore : 6.752709268799811
- fscore : 0.2545845515075978 	 maeOverFscore : 6.752907726649149
- fscore : 0.2545843704403055 	 maeOverFscore : 6.702911044140701
- fscore : 0.25427924623684933 	 maeOverFscore : 6.7471879931893675

# Ridge alpha 10.0 SelectK 8 RobustScaler
- fscore : 0.2749365314732829 	 maeOverFscore : 6.261598197378067
- fscore : 0.2752106860272466 	 maeOverFscore : 6.2545566924636615
- fscore : 0.27483218889350364 	 maeOverFscore : 6.215133163802377
- fscore : 0.2744506712189898 	 maeOverFscore : 6.258121761417609

# Ridge alpha 10.0 SelectK 8 PCA 2 dim
- fscore : 0.24107474262012732 	 maeOverFscore : 8.224389019645066
- fscore : 0.2410970325797045 	 maeOverFscore : 8.222939301354975
- fscore : 0.24133791359125045 	 maeOverFscore : 8.159670908854318
- fscore : 0.2408981513939592 	 maeOverFscore : 8.21528441818803

# Ridge alpha 10.0 SelectK 8 PCA 1 dim
- 0.16343634206703112 12.186417405981281
- 0.16353559251460106 12.177879886692459
- 0.16345101377437446 12.102961373810988
- 0.16322487488861562 12.179870422653915

# Train

In [16]:
clf = Ridge(alpha=10.0)
scaler= RobustScaler()

x = scaler.fit_transform(X)
clf.fit(x, Y)

Ridge(alpha=10.0, copy_X=True, fit_intercept=True, max_iter=None,
      normalize=False, random_state=None, solver='auto', tol=0.001)

# Submit

In [24]:
def submit(clf, name, features=None, scaler=None):
    x_test = load('test')
    x_test = reshape(x_test)
    
    if features is not None:
        x_test = x_test[:, features]
    
    if scaler is not None:
        x_test = scaler.transform(x_test)
        print("transform")
    
    pred = clf.predict(x_test)

    submission = pd.read_csv(os.path.join(data_path, 'sample_submission.csv'))
    submission.iloc[:, 1:] = pred.reshape(-1, 1600)

    submission.to_csv(os.path.join(submit_path, f'{name}.csv'), index = False)

In [27]:
submit(clf, 'Ridge_10_SelectK_8_Robust', selectK_8, scaler)

transform


- https://dacon.io/competitions/official/235591/mysubmission/
- D:\인공지능_공모전\github\submit